In [1]:
#import usual packages for data manipulation
import pandas as pd
import numpy as np

#import usual packages for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import datetime to get the current date
import datetime as dt

#from alpha vantage, import the TimeSeries class
from alpha_vantage.timeseries import TimeSeries

import json

#set the style of the plots
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
#set the API key
my_key = 'DJ3QKKTFZ5J298QY'

#initialize the FundamentalData class
ts = TimeSeries(key=my_key, output_format='pandas')

In [3]:
#choose a ticker
ticker = 'AAPL'

In [4]:
stock_data, stock_meta_data = ts.get_daily(ticker,outputsize='full')

In [5]:
df = stock_data.copy()
df.columns = ['open','high','low','close','volume']
df.sort_index(inplace=True)
df = df[df.index.year >= 2014]
df.to_csv(f'../data/{ticker}_stock.csv')

In [6]:
def wtd_std(vals,wts):
    wtd_mean = np.average(vals,weights=wts)
    vals_demeaned = vals - wtd_mean
    wtd_var = np.average(vals_demeaned**2,weights=wts)
    return np.sqrt(wtd_var)

def get_wtd_std(vals,wts,window):
    ser = np.zeros(len(df))
    for i in range(len(df)-1 , 0, -1):
        if i < window:
            ser[i] = np.nan
        else:
            ser[i] = wtd_std(vals.iloc[i-window+1:i+1],wts.iloc[i-window+1:i+1])
    return ser

In [7]:
#window sizes for moving avgs and volume-weighted moving avgs
windows = [5, 11, 21, 63, 125, 251]
#calculate past moving averages, standard dev's, volume-weighted moving averages, and standard dev's of volume-weighted moving averages
for w in windows:
    df[f'ma_{w}'] = df['close'].rolling(w).mean()
    df[f'std_{w}'] = df['close'].rolling(w).std()
    #compute the w-day volume weights
    df[f'wts_{w}'] = df['volume'] / df['volume'].rolling(w).sum()
    #compute VWAP by dotting the w-day close price with the w-day volume weights
    df[f'vwap_{w}'] = (df['close'] * df[f'wts_{w}']).rolling(w).sum()
    #compute the weighted standard deviation of the volume-weighted moving averages
    df[f'vwstd_{w}'] = get_wtd_std(vals = df[f'close'],
                                   wts = df[f'wts_{w}'],
                                   window = w)
    #drop the volume weights
    df.drop(columns=f'wts_{w}',inplace=True)
    # add rolling bollinger bands for windows w in [5,11,21,63,125,251]
    # bollinger bands are defined as the moving average plus/minus 2 times the moving standard deviation
    # store the upper and lower bands in columns f'bbu_{w}' and f'bbl_{w}'
    # do the same for the vol-weighted averages and standard deviations
    df[f'bbu_{w}'] = df[f'ma_{w}'] + 2 * df[f'std_{w}']
    df[f'bbl_{w}'] = df[f'ma_{w}'] - 2 * df[f'std_{w}']
    df[f'vwbbu_{w}'] = df[f'vwap_{w}'] + 2 * df[f'vwstd_{w}']
    df[f'vwbbl_{w}'] = df[f'vwap_{w}'] - 2 * df[f'vwstd_{w}']
#drop NaN's
# df.dropna(inplace=True)
#save to csv in same folder as the original data, with title f'{ticker}_stock_features.csv'
df.to_csv(f'../data/{ticker}_stock_features.csv')
    